## Racktrack Shipments

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

from datetime import date, datetime, timedelta
from pandas import DataFrame

In [2]:
# Getting credentials from another file
import creds_panoply

POSTGRES_ADDRESS = creds_panoply.address
POSTGRES_PORT = creds_panoply.port
POSTGRES_USERNAME = creds_panoply.username
POSTGRES_PASSWORD = creds_panoply.pw
POSTGRES_DBNAME = creds_panoply.dbname

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [3]:
# Getting relevant data from Panoply

n_days = 4                 # Number of past days for which we'll like to get the data for
date_cutoff = str(date.today() - timedelta(n_days))
# date_cutoff = '2021-05-25'

tbl_name = "public.racetracks_shipped_items_by_date"
query = "SELECT * FROM " + tbl_name + " WHERE shipping_date > '" + date_cutoff + "';"
df = pd.read_sql_query(query, cnx)
df.head()

,shipping_date,fulfillment_sku,fulfillment_sku_variant,shipped_quantity
0,2021-05-31,BASE-BREWCLS,None,0
1,2021-05-31,BASE-BREWCREBRL,None,0
2,2021-05-31,BASE-BREWCRM,None,0
3,2021-05-31,BASE-BREWHZL,None,0
4,2021-05-31,BASE-BREWMCH,None,0


In [4]:
# Adding columns for Mfg. SKUs & sales_prices
df['Mfg_SKU'] = "TBC"
df['Avg_Sales_Price'] = 0
df.head()


,shipping_date,fulfillment_sku,fulfillment_sku_variant,shipped_quantity,Mfg_SKU,Avg_Sales_Price
0,2021-05-31,BASE-BREWCLS,None,0,TBC,0
1,2021-05-31,BASE-BREWCREBRL,None,0,TBC,0
2,2021-05-31,BASE-BREWCRM,None,0,TBC,0
3,2021-05-31,BASE-BREWHZL,None,0,TBC,0
4,2021-05-31,BASE-BREWMCH,None,0,TBC,0


In [5]:
# Mapping Table
df_mapping = pd.read_excel('SKUMapping_Fulfillment_to_Mfg.xlsx', sheet_name='sku_mapping')
df_mapping.head()

,Fulfillment_SKU,Variant,Mfg_SKU,base_sales_price
0,BASE-BREWCLS,NaN,BREWCLS,3.1
1,BASE-BREWCREBRL,NaN,BREWCREBRL,3.1
2,BASE-BREWCRM,NaN,BREWCRM,3.1
3,BASE-BREWHZL,NaN,BREWHZL,3.1
4,BASE-BREWMCH,NaN,BREWMCH,3.1


In [6]:
for i in df.index:
    # cartridges
    if df.loc[i, 'fulfillment_sku_variant'] == None:
        for j in df_mapping.index:
            if df.loc[i, 'fulfillment_sku'] == df_mapping.loc[j, 'Fulfillment_SKU']:
                df.loc[i, 'Mfg_SKU'] = df_mapping.loc[j, 'Mfg_SKU']
                df.loc[i, 'Avg_Sales_Price'] = df_mapping.loc[j, 'base_sales_price']
    # Non-cartridges
    else:
        for j in df_mapping.index:
            if (df.loc[i, 'fulfillment_sku'] == df_mapping.loc[j, 'Fulfillment_SKU']) and (df.loc[i, 'fulfillment_sku_variant'] == df_mapping.loc[j, 'Variant']):
                df.loc[i, 'Mfg_SKU'] = df_mapping.loc[j, 'Mfg_SKU']
                df.loc[i, 'Avg_Sales_Price'] = df_mapping.loc[j, 'base_sales_price']

df.head()
        

,shipping_date,fulfillment_sku,fulfillment_sku_variant,shipped_quantity,Mfg_SKU,Avg_Sales_Price
0,2021-05-31,BASE-BREWCLS,None,0,BREWCLS,3.1
1,2021-05-31,BASE-BREWCREBRL,None,0,BREWCREBRL,3.1
2,2021-05-31,BASE-BREWCRM,None,0,BREWCRM,3.1
3,2021-05-31,BASE-BREWHZL,None,0,BREWHZL,3.1
4,2021-05-31,BASE-BREWMCH,None,0,BREWMCH,3.1


In [7]:
# Dropping rows with shipped_quantity = 0
df = df[df.shipped_quantity > 0]

In [8]:
# Export df to excel

import platform
import os
os_name = platform.system()


dateTimeObj = datetime.now(tz=None)
timeStamp = str(str(dateTimeObj.month) + "." + str(dateTimeObj.day) + "." + str(dateTimeObj.year))
filename = "Test_df_" + timeStamp + ".xlsx"

if os_name == "Windows":
    desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    out_path = desktop_path + "//" + filename
    with pd.ExcelWriter(out_path) as writer:
        df.to_excel(writer, sheet_name='Summary')

In [9]:
# Dropping some columns
df.drop(columns=['fulfillment_sku', 'fulfillment_sku_variant'], inplace=True)

# Dropping rows with shipped_quantity = 0
df = df[df.shipped_quantity > 0]

# Reordering columns
df = df[['shipping_date', 'Mfg_SKU', 'shipped_quantity', 'Avg_Sales_Price']]



In [10]:
# Output
output = df.groupby(['shipping_date', 'Mfg_SKU']).sum()
output.head()

shipped_quantity  Avg_Sales_Price
shipping_date Mfg_SKU                                      
2021-06-01    BOTL12                   126             10.0
              BOTL22                  1654             13.0
              BREWCLS                   77              3.1
              BREWCREBRL                17              3.1
              BREWCRM                  967              3.1

In [11]:
# Exporting the output  to Excel

import platform
import os
os_name = platform.system()


dateTimeObj = datetime.now(tz=None)
timeStamp = str(dateTimeObj.strftime("%m.%d.%Y %I-%M %p"))
filename = "Daily_Racetrack_Shipments_" + timeStamp + ".xlsx"

if os_name == "Darwin":
    desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Downloads')
    out_path = desktop_path + "/" + filename
    with pd.ExcelWriter(out_path) as writer:
        output.to_excel(writer, sheet_name='Summary')

if os_name == "Windows":
    desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Downloads')
    out_path = desktop_path + "//" + filename
    with pd.ExcelWriter(out_path) as writer:
        output.to_excel(writer, sheet_name='Summary')
